In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Load the CSV file
data = pd.read_csv('data_ANN.csv')

# Extract features (AT, V, AP, RH) into X
X = data[['AT', 'V', 'AP', 'RH']].values

# Extract target (PE) into y
y = data[['PE']].values

# Normalize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std  # Standardization

# Normalize target values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std  # Standardization

# Define the neural network architecture
input_size = 4    # Number of input neurons (four features)
hidden_size = 5   # Number of neurons in the hidden layer
output_size = 1   # Number of output neurons (one output)

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size)  # Weights for input to hidden layer
b1 = np.zeros((1, hidden_size))                # Bias for hidden layer
W2 = np.random.randn(hidden_size, output_size) # Weights for hidden to output layer
b2 = np.zeros((1, output_size))                # Bias for output layer

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Forward propagation
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = Z2  # Linear activation for output
    return Z1, A1, Z2, A2

# Loss function
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = np.mean((y_true - y_pred) ** 2)  # Mean Squared Error for regression
    return loss

# Backpropagation
def backpropagation(X, y_true, Z1, A1, Z2, A2):
    m = X.shape[0]
    dZ2 = (A2 - y_true)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

# Update parameters
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# Train the neural network
def train(X, y, epochs, learning_rate):
    global W1, b1, W2, b2
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X)
        loss = compute_loss(y, A2)
        if(epoch%500==0):
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss}")
        dW1, db1, dW2, db2 = backpropagation(X, y, Z1, A1, Z2, A2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    return W1, b1, W2, b2

# Make predictions
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X)
    return A2

# Train the neural network
W1, b1, W2, b2 = train(X, y, epochs=10000, learning_rate=0.01)

# Make predictions
predictions = predict(X, W1, b1, W2, b2)

# Denormalize the predictions
denormalized_predictions = predictions * y_std + y_mean

# Compute R² score
r2 = r2_score(data[['PE']], denormalized_predictions)

print("Predictions:")
print(denormalized_predictions)
print(f"R² Score: {r2}")


Epoch 1/10000 - Loss: 0.6140998472852384
Epoch 501/10000 - Loss: 0.1797607692899296
Epoch 1001/10000 - Loss: 0.15340640009048326
Epoch 1501/10000 - Loss: 0.14047181496768998
Epoch 2001/10000 - Loss: 0.1313519934812943
Epoch 2501/10000 - Loss: 0.12407481085388179
Epoch 3001/10000 - Loss: 0.11803407867780254
Epoch 3501/10000 - Loss: 0.11294665871653273
Epoch 4001/10000 - Loss: 0.10863111059716238
Epoch 4501/10000 - Loss: 0.10495356833157321
Epoch 5001/10000 - Loss: 0.10181005700691899
Epoch 5501/10000 - Loss: 0.09911805352579878
Epoch 6001/10000 - Loss: 0.09681118123456309
Epoch 6501/10000 - Loss: 0.09483545489620457
Epoch 7001/10000 - Loss: 0.09314652912642318
Epoch 7501/10000 - Loss: 0.09170766761943334
Epoch 8001/10000 - Loss: 0.09048823939333099
Epoch 8501/10000 - Loss: 0.08946259682183365
Epoch 9001/10000 - Loss: 0.0886092271263084
Epoch 9501/10000 - Loss: 0.08791009896601136
Predictions:
[[470.15794167]
 [441.62469971]
 [478.53803942]
 ...
 [434.43114159]
 [439.39530823]
 [448.9285